In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from ast import literal_eval 
import pyspark.sql.functions as F
from pyspark.sql.functions import udf

In [ ]:
import numpy as np
import pandas as pd

In [3]:
from pyspark.sql.types import StringType, IntegerType, ArrayType
from pyspark.sql.functions import regexp_replace,col,array_contains,explode
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id
from functools import reduce
from pyspark.sql.functions import concat_ws

In [ ]:
#loading train data to pandas dataframe and dumping it to spark dataframe
train_df = pd.read_csv(r'train.csv')
train_spark_df = spark.createDataFrame(train_df)

In [ ]:
#loading label data file that is generated initially to pandas dataframe and dumping it to spark dataframe
lables_df = pd.read_csv(r'genres.csv')
lables_spark_df = spark.createDataFrame(lables_df)

In [5]:
#loading test data to pandas dataframe and dumping it to spark dataframe
test_pd_df = pd.read_csv(r'test.csv')
test_spark_df = spark.createDataFrame(test_pd_df)

In [6]:
#using joins to combine both data frame with using row_id as key and dumped it into new dataframe
x1 = train_spark_df.withColumn("row_id", monotonically_increasing_id())
x2 = lables_spark_df.withColumn("row_id", monotonically_increasing_id())
train_df = x1.join(x2, "row_id").drop("row_id")

In [ ]:
#using joins adding label columns to test data set
x3 = test_spark_df.withColumn("row_id", monotonically_increasing_id())
test_df = x3.join(x2, "row_id").drop("row_id")

In [7]:
train_df.show(1)

+--------+----------+--------------------+---------+--------+--------+--------------+----------+--------+--------------+---------------+--------+-----------------+-------+------------------+-----------+-------------+------------+----------------+-----------+---------+-----------------+---------+----------------+
|movie_id|movie_name|                plot|    genre|# Drama | Comedy | Romance Film | Thriller | Action | World cinema | Crime Fiction | Horror | Black-and-white | Indie | Action/Adventure | Adventure | Family Film | Short Film | Romantic drama | Animation | Musical | Science Fiction | Mystery | Romantic comedy|
+--------+----------+--------------------+---------+--------+--------+--------------+----------+--------+--------------+---------------+--------+-----------------+-------+------------------+-----------+-------------+------------+----------------+-----------+---------+-----------------+---------+----------------+
|18549958|Love, Mary|Mary was a lonely...|['Drama']|      

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StopWordsRemover , CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import HashingTF, IDF

In [8]:
%%time
#adding regex tokenizer for splitting into words
regex=RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
#adding own defined set of words
add_stopwords=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
#removing stopwords
stopwordsRemover=StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
#adding hashingTF to extract term doc values
hashingTF=HashingTF(inputCol="filtered",outputCol="rawfeatures")
#adding idf to extract idf values
idf=IDF(inputCol='rawfeatures',outputCol='features',minDocFreq=5)
#adding all these to pipeline
pipeline=Pipeline(stages=[regex, stopwordsRemover, hashingTF,idf])
#applying this pipeline on train dataframe
op1=pipeline.fit(train_df)
train_dataset=op1.transform(train_df)
#applying this pipeline on test dataframe
op2=pipeline.fit(test_df)
test_dataset=op2.transform(test_df)

CPU times: user 151 ms, sys: 46.2 ms, total: 197 ms
Wall time: 47.7 s


In [9]:
train_dataset.select('features').show(5)

+--------------------+
|            features|
+--------------------+
|(262144,[14699,16...|
|(262144,[9129,120...|
|(262144,[3834,420...|
|(262144,[1133,183...|
|(262144,[571,2089...|
+--------------------+
only showing top 5 rows



In [10]:
%%time
from pyspark.ml.classification import LogisticRegression
finallist=[]
labels=lables_spark_df.columns
lr=LogisticRegression(featuresCol='features',maxIter=700)
#iterating through lr model for 20 labels and storing predictions in list
for label in labels:
    lr.setLabelCol(label)
    lrModel=lr.fit(train_dataset)
    predictions=lrModel.transform(test_dataset)
    predictions=predictions.withColumn("prediction",F.col("prediction").cast(IntegerType()))
    finallist.append(predictions.select("movie_id","prediction"))

CPU times: user 14.5 s, sys: 4.76 s, total: 19.3 s
Wall time: 1h 1min 50s


In [12]:
%%time
#after making predictions which are stored in finallist now these predictions are appeneded to movie id
df=[j.selectExpr('movie_id', f'prediction as prediction_{i}') for i,j in enumerate(finallist)]
temp = reduce(lambda x, y: x.join(y, ['movie_id'], how='full'), df)
col = ['prediction_%d' % i for i in range(len(finallist))]
#appended movie id and predictions are now dumped to .csv file
temp= temp.withColumn('predictions',concat_ws(" ",*col)).drop(*col).toPandas().to_csv("part2.csv",index=False)

CPU times: user 632 ms, sys: 226 ms, total: 859 ms
Wall time: 3min 53s
